# Trabajo práctico Minería de Textos 2024


---


## Postgrado de especialización en Inteligencia de Datos Orientada a BigData.
Facultad de Informática. Universidad Nacional de La Plata (UNLP)


---


*Alumno* : **Diego Fernando Tarrío**

*DNI* : **22151919**



**Trabajo**: Se procede a implementar el procesamiento de texto que representan e-mails recibidos por una compañía de seguros a su sector de atención al cliente.

Se realizará un trabajo similar a Named Entity Recognition (NER) del mismo, pero que no requiere fine tuning, denominado "Prompt-Based Structured Extraction", dotando al modelo de la capacidad de determinar las entidades correspondientes a un modelo de dominio relacionado al sector de seguros (Entidades como Póliza, Riesgos, Coberturas, Clientes serán utilizadas). Esta capacidad de determinar las entidades serán dadas al modelo a través de un mecanismo básico de Retrieval Augmented Generation (RAG). Se evaluó la alternativa de Fine Tuning del modelo, pero en el uso de LLM considero mas oportuno evitar costo de entrenamiento aprovechando el envío de información directamente desde el Prompt como con RAG (La pequeña penalidad de performance en tiempos de respuesta son despreciables para el uso pretendido). Un Fine Tuning implicaría reentrenar al modelo ajustando los pesos y para esta actividad específica no lo consideré justificado.

Adicionalmente, se procederá a generar respuestas automáticas al cliente (En base a análisis de sentimientos, evaluando si se trata de un reclamo o de un pedido de gestión). En el caso de pedido de gestión, una vez identificados los atributos y entidades, se procederá a elaborar una estructura JSon que permita utilizarse en una invocación a una API Rest de un producto interno de la compañía (Ej: API para alta de póliza).

 Esto permitiría aprovechar la IA para automatizar la gestión de atención al cliente con respuestas según análisis de sentimientos y propuestas de acciones directas al producto de la compañía mediante la automatización de invocaciones a la capa API Rest del software de gestión de la cía, según las intenciones expresadas por el cliente en su e-mail.

El modelo LLM a utilizar será **Claude 3.5** , se requiere una API-Key que será provista en un archivo .env que deberá subirse manualmente (por favor, moderar su uso ya que cada invocación consume algunos centavos de u$s prepagos). Puede utilizarse un wrapper del modelo para que esta solución sea agnóstica del mismo, pudiendo variar el modelo LLM de elección por ejemplo a OpenAI con su modelo GPT.


### Observaciones:

Hay varios puntos posibles de mejoras sobre este trabajo, sólo cito algunos:



*   Para realmente contar con un mecanismo completo de RAG, se podría contar con una base de datos vectorial que guarde embeddings por ejemplo de valores posibles de los atributos de entidades, como por ejemplo los nombres de coberturas posibles, para luego desde la extracción de datos desde el prompt, se podría generar el embedding de la cobertura expresada en el mail y buscar un nombre de cobertura válido en el producto de la compañía.
*   Las emulaciones a invocación a APIs de la compañía se limitan a generar un JSon con entidades y valores de atributos extraídos. Podría directamente delegarse al modelo que genere el código para la invocación a una API.
*   Podría realizarse una alternativa a RAG mediante fine tuning y NER, con el costo de reentrenar el modelo (Esto no lo permite directamente GPT ni Claude, pero podrían utilizarse otros modelos alternativos que sí permitan fine-tuning) Esto permitiría realizar comparaciones de resultados y performance de los distintos enfoques.
*  El modelo se encuentra HardCoded, podría estar dentro de un wrapper que permita variar el modelo LLM a utilizar, pudiendo generar misma experiencia tanto en Claude como en GPT u otro similar.



In [ ]:
#Instalo la librería correspondiente a la empresa anthropic para el uso del LLM Claude
!pip install anthropic
!pip install python-dotenv

import json
import anthropic
import os

#IMPORTANTE: Se debe agregar el archivo .env con la API_KEY correspondiente a anthropic para que pueda invocarse la api del modelo LLM Claude
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv()) #lee local .env file a ubicarse en el directorio raiz

from pprint import pprint


In [ ]:
#La siguiente clase contiene al modelo y los métodos para invocación al mismo, además del método para extracción de atributos y entidades, junto al análisis de sentimientos
#adicional al método que emula la invocación a APIS del producto core de la compañía

class RAGEntityMapper:
    def __init__(self, api_key=None):

        # Si no se proporciona API key, intenta cargarla de variables de entorno
        self.client = anthropic.Anthropic(
            api_key=api_key or os.getenv('ANTHROPIC_API_KEY')
        )

        # Mapeo de entidades del modelo relacional, se establecen sólo 3 entidades con una cantidad muy reducida de atributos a extraer del texto del mail:
        self.entity_schema = {
            "Poliza": {
                "attributes": ["numero", "tipo", "fecha_inicio", "fecha_fin", "cliente_id", "Premio"],
                "relations": ["Cliente", "Cobertura"]
            },
            "Cliente": {
                "attributes": ["nombre", "documento", "email", "telefono"],
                "relations": ["Poliza"]
            },
            "Cobertura": {
                "attributes": ["nombre"],
                "relations": ["Poliza"]
            },
        }

    def call_claude_api(self, prompt):
        try:
            response = self.client.messages.create(
                model="claude-3-opus-20240229",  # Se puede cambiar por otro modelo
                max_tokens=1000,
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            )

            # Extraigo el texto de la respuesta
            return response.content[0].text

        except Exception as e:
            print(f"Error en la llamada a Claude: {e}")
            return json.dumps({})  # Devuelvo JSON vacío en caso de error


    def extract_entities(self, email_text):

        prompt = f"""
        Realizar dos tareas con el siguiente email:

        TEXTO: "{email_text}"

        TAREA 1 - EXTRACCIÓN DE ENTIDADES:
        ESQUEMA DE ENTIDADES:
        {json.dumps(self.entity_schema, indent=2)}

        INSTRUCCIONES EXTRACCIÓN:
        1. Extrae solo los datos que encuentres explícitamente en el email en el esquema de entidades.
        2. A excepción de fechas, si se puede inferir fecha de fin en base a la de inicio de vigencia, calcularla y devolverla en formato ISO.
        3. Si no encontrás un atributo, dejarlo como null
        4. Responder en formato JSON estricto
        5. Si no hay información, devolver un JSON vacío

        TAREA 2 - ANÁLISIS DE SENTIMIENTO:
        1. Clasifica el tono del email en:
           - Neutro (consulta informativa)
           - Positivo (interés en contratar)
           - Negativo (reclamo o insatisfacción)
        2. Si es un reclamo, identifica el motivo principal
        3. Genera una respuesta apropiada según el tono detectado emulando la respuesta de un call center / customer care por parte de la Compañía de seguros

        FORMATO RESPUESTA JSON:
        {{
          "Entidades": {{
          {{
              "Poliza": {{
                  "numero": null,
                  "tipo": null,
                  "fecha_inicio": null,
                  "fecha_fin": null,
                  "cliente_id": null,
                  "Premio": null
              }},
              "Cliente": {{
                  "nombre": null,
                  "documento": null,
                  "email": null,
                  "telefono": null
              }},
              "Cobertura": {{
                  "nombre": null
              }}
          }}
          }},
          "Sentimiento": {{
              "tono": "neutro/positivo/negativo",
              "motivo_reclamo": null o descripción breve,
              "respuesta_sugerida": "texto de respuesta"
          }}
        }}
        """

        # Llamada a Claude para extracción
        response = self.call_claude_api(prompt)

        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print("No se pudo parsear la respuesta de Claude")
            return {}

    def prepare_api_payload(self, extracted_data):
        # Filtro valores no nulos
        payload = {}
        for entity, attributes in extracted_data.items():
            entity_payload = {
                k: v for k, v in attributes.items()
                if v is not None
            }
            if entity_payload:
                payload[entity] = entity_payload

        return payload

    def invoke_apis(self, payload):
        # Lógica mockeada para invocar APIs según entidades
        results = {}
        #en lugar de invocar a las apis muestro lo que se construyó en pantalla:
        print("Entidades y atributos identificados según el schema brindado:")
        pprint(payload)
##        if "Cliente" in payload:
##            results["Cliente"] = self.cliente_api.create(payload["Cliente"])
##
##        if "Poliza" in payload:
##            # Asegurar dependencias
##            if "Cliente" in results:
##                payload["Poliza"]["cliente_id"] = results["Cliente"].get("id")
##
##            results["Poliza"] = self.poliza_api.create(payload["Poliza"])
##
##        return results



Genero un archivo de ejemplos de mails con diferentes pedidos/reclamos/comentarios

In [ ]:

def generar_emails_ejemplo(archivo_salida='emails_ejemplos.txt'):
    """
    Genera un archivo con varios ejemplos de emails para pruebas
    """
    emails_ejemplos = [
        """
From: juan.perez@empresa.com
To: empresa@seguros.com
Subject: Consulta Seguro Responsabilidad Civil
Body:
Buenas tardes, soy Juan Pérez, con documento 30456789, cel: 1156782345.
Quiero contratar un seguro de responsabilidad civil
para mi empresa, con cobertura RC completa,
a partir del día 15 de diciembre 2024 por el término de 6 meses.
""",
        """
From: maria.gonzalez@empresa.com
To: empresa@seguros.com
Subject: Reclamo por Póliza automotor
Body:
Este ya es mi tercer reclamo y hasta ahora no obtuve respuesta satisfactoria. Choqué con mi auto con seguro nro POL-2023-5678 y no me indican si la cobertura cubre la reparación. Necesito una explicación urgente.
Mi teléfono de contacto es 1187654321.
""",
        """
From: carlos.rodriguez@empresa.com
To: empresa@seguros.com
Subject: Nueva Cotización de Seguro
Body:
Estoy interesado en un seguro para mi auto Honda Civic 1.8 Exs Mt 140cv.
Quiero una cobertura amplia de Terceros completo, que incluya cobertura por granizo.
Pueden contactarme al 1165432198 o a mi mail
Saludos, Carlos Rodríguez.
""",
        """
From: noelia.sanchez@empresa.com
To: empresa@seguros.com
Subject: Reclamo resuelto
Body:
Buenas tardes, quería dejar constancia de mi agradecimiento a la ejecutiva de mi cuenta Camila Perez por su rápida gestión ante el reclamo que realicé la semana pasada por datos incorrectos en la póliza emitida. Me resolvió el problema en tiempo record.
Saludos y nuevamente gracias! Noelia.
"""
    ]

    # Escribo los ejemplos en un archivo
    with open(archivo_salida, 'w', encoding='utf-8') as f:
        for email in emails_ejemplos:
            f.write(email + "\n---SEPARADOR---\n")

    print(f"Archivo de emails de ejemplo generado: {archivo_salida}")

def procesar_emails_desde_archivo(extractor, archivo_entradas='emails_ejemplos.txt'):
    """
    Lee emails desde un archivo y procesa cada uno con RAGEntityMapper
    """
    # Verifico si el archivo existe
    if not os.path.exists(archivo_entradas):
        print(f"El archivo {archivo_entradas} no existe. Generando ejemplos...")
        generar_emails_ejemplo(archivo_entradas)

    # Leo el archivo de emails
    with open(archivo_entradas, 'r', encoding='utf-8') as f:
        contenido = f.read()

    # Separo los emails según el set de caracteres separadores elegidos
    emails = contenido.split("---SEPARADOR---")

    # Proceso cada email iterando la lista de emails
    resultados = []
    for i, email in enumerate(emails, 1):
        if email.strip():  # Ignoro emails vacíos
            print(f"\n{'='*50}")
            print(f"Procesando Email {i}:")
            print(f"{'='*50}")

            try:
                # Extracción de entidades
                extracted_data = extractor.extract_entities(email)

                # Preparo payload
                payload = extractor.prepare_api_payload(extracted_data)

                # Invoco APIs y obtengo resultado
                resultado = extractor.invoke_apis(payload)

                # Guardo resultados
                resultados.append({
                    'email_numero': i,
                    'email': email,
                    'extracted_data': extracted_data,
                    'payload': payload,
                    'resultado': resultado
                })

            except Exception as e:
                print(f"Error procesando email {i}: {e}")

    return resultados



In [ ]:
def main():

    # Creo instancia del extractor
    claude_api_key = os.getenv('ANTHROPIC_API_KEY')
    print(claude_api_key)

    extractor = RAGEntityMapper(api_key=claude_api_key)

    # Genero emails de ejemplo (opcional, si no existen)
    generar_emails_ejemplo()

    # Proceso emails desde archivo
    resultados = procesar_emails_desde_archivo(extractor)

    # Opcional: Guardar resultados en un archivo JSON para análisis posterior
    import json
    with open('resultados_procesamiento.json', 'w', encoding='utf-8') as f:
        json.dump(resultados, f, indent=2, ensure_ascii=False)

    print("\nProcesamiento completado. Resultados guardados en 'resultados_procesamiento.json'")

# Ejecuto el script
if __name__ == "__main__":
    main()